In [1]:
from dotenv import load_dotenv
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_core.messages import AIMessage,HumanMessage
from langchain_core.tools import tool
import os
import requests

EXCHANGE_RATE_API_KEY = os.getenv('EXCHANGE_RATE_API_KEY')

C:\Users\ssahu\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()
llm = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-7B-Instruct", #google/gemma-2-2b-it
    task="conversational"

)

model = ChatHuggingFace(llm=llm)

In [3]:
@tool
def get_conversion_factor(base_currency : str,target_currency: str) -> float:
    '''
    This function fetches the currency conversion factor between a given base currency and target currency
    '''
    url = f'https://v6.exchangerate-api.com/v6/{EXCHANGE_RATE_API_KEY}/pair/{base_currency}/{target_currency}'
    response =requests.get(url)
    return response.json()

In [4]:
get_conversion_factor.invoke({
    'base_currency': 'USD',
    'target_currency' : 'INR'
})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1764979201,
 'time_last_update_utc': 'Sat, 06 Dec 2025 00:00:01 +0000',
 'time_next_update_unix': 1765065601,
 'time_next_update_utc': 'Sun, 07 Dec 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 90.0069}

In [5]:
@tool
def convert(base_currency_value: int,conversion_rate: float) -> float:
    '''
    Given currency conversion rate this function calculates the target currency value from a given base currency value
    '''
    return base_currency_value*conversion_rate

In [6]:
convert.invoke({
    'base_currency_value' : 10,
    'conversion_rate' : 90.0069
})

900.069

In [7]:
#Tool binding
llm_with_tools = model.bind_tools([get_conversion_factor,convert])

In [8]:
llm_with_tools.invoke('Hello')

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 338, 'total_tokens': 348}, 'model_name': 'Qwen/Qwen2.5-7B-Instruct', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--851a359a-ef12-4888-bd94-17c73356d2a9-0', usage_metadata={'input_tokens': 338, 'output_tokens': 10, 'total_tokens': 348})

In [9]:
messages = [HumanMessage('What is the conversion factor between USD and INR and based on that can you convert 10 usd to inr')]

In [10]:
messages

[HumanMessage(content='What is the conversion factor between USD and INR and based on that can you convert 10 usd to inr', additional_kwargs={}, response_metadata={})]

In [11]:
ai_message = llm_with_tools.invoke(messages)

In [12]:
ai_message.tool_calls   
#See here instead of calling the tools in sequence it calls both the tool at a time
#So here you can see conversion_rate is different as it takes from its own knowledge
#But our aim is to call get_conversion_factor first then give the results to convert tool
#So for that we can use injected tool

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': 'call_n5npsqs1o3n8h3eahgniobpj',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 10, 'conversion_rate': 74.85},
  'id': 'call_ocg4ai90xm8ewjr72ctu8f9s',
  'type': 'tool_call'}]

In [13]:
from langchain_core.tools import InjectedToolArg
from typing import Annotated

#Here we are giving info to llm that
# LLM do not try to fill this argument I (the devloper/runtime) will inject this value after running earlier tools

@tool
def convert(base_currency_value: int,conversion_rate: Annotated[float,InjectedToolArg]) -> float:
    '''
    Given currency conversion rate this function calculates the target currency value from a given base currency value
    '''
    return base_currency_value*conversion_rate


In [ ]:
llm_with_tools = model.bind_tools([get_conversion_factor,convert])

In [15]:
import json

for tool_call in ai_message.tool_calls:
    # Exevute the 1st tool and get the value of conversion rate
    if tool_call['name'] == 'get_conversion_factor':
        tool_message1 = get_conversion_factor.invoke(tool_call)
        print(tool_message1)
        #Fetch this conversion rate
        conversion_rate = json.loads(tool_message1.content)['conversion_rate']
        #append this tool message to messages list
        messages.append(tool_message1)
        # Execute the 2nd tool using the conversion rate from tool1
    if tool_call['name'] == 'convert':
        #fetch the current arg
        tool_call['args']['conversion_rate'] = conversion_rate
        tool_message2= convert.invoke(tool_call)
        messages.append(tool_message2)

content='{"result": "success", "documentation": "https://www.exchangerate-api.com/docs", "terms_of_use": "https://www.exchangerate-api.com/terms", "time_last_update_unix": 1764979201, "time_last_update_utc": "Sat, 06 Dec 2025 00:00:01 +0000", "time_next_update_unix": 1765065601, "time_next_update_utc": "Sun, 07 Dec 2025 00:00:01 +0000", "base_code": "USD", "target_code": "INR", "conversion_rate": 90.0069}' name='get_conversion_factor' tool_call_id='call_n5npsqs1o3n8h3eahgniobpj'


In [16]:
messages

[HumanMessage(content='What is the conversion factor between USD and INR and based on that can you convert 10 usd to inr', additional_kwargs={}, response_metadata={}),
 ToolMessage(content='{"result": "success", "documentation": "https://www.exchangerate-api.com/docs", "terms_of_use": "https://www.exchangerate-api.com/terms", "time_last_update_unix": 1764979201, "time_last_update_utc": "Sat, 06 Dec 2025 00:00:01 +0000", "time_next_update_unix": 1765065601, "time_next_update_utc": "Sun, 07 Dec 2025 00:00:01 +0000", "base_code": "USD", "target_code": "INR", "conversion_rate": 90.0069}', name='get_conversion_factor', tool_call_id='call_n5npsqs1o3n8h3eahgniobpj'),
 ToolMessage(content='900.069', name='convert', tool_call_id='call_ocg4ai90xm8ewjr72ctu8f9s')]

In [18]:
llm_with_tools.invoke(messages).content

'Based on the latest conversion rate provided, 1 USD is approximately equal to 90.0069 INR. Therefore, 10 USD would be approximately equal to 900.069 INR.'